In [1]:
# Get raw data
with open('input/10.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [3]:
# Part 1
def get_buttons(x,y):
    buttons = np.zeros((len(x), y), dtype=int)
    buttons[*zip(*[[i,k] 
                   for i,j in enumerate(x) 
                   for k in eval(f'[{j[1:-1]}]')])] = 1
    return buttons

manual = [[(z:=np.array([[int(k=='#') for k in j[0][1:-1]]])), 
           get_buttons(j[1:-1], z.shape[1]), 
           np.array([int(k) for k in j[-1][1:-1].split(',')])]
          for i in rawinput.split('\n') 
          for j in [i.split()]]

hlist = lambda x: x.astype('<U1').view(f'<U{x.shape[1]}').reshape(-1).tolist()

tot_n_press = 0
for req, buttons, _ in manual:
    poss = np.zeros_like(req)
    hist = hlist(poss)
    n_press = 0
    while True:
        n_press += 1
        poss = np.logical_xor(np.repeat(poss, buttons.shape[0], axis=0),
                              np.tile(buttons, (poss.shape[0], 1))).astype(int)
        if np.any(np.all(poss == req, axis=1)):
            break
        poss = poss[((h:=np.array(hlist(poss)))[s:=np.lexsort((np.arange(h.shape[0]), h))] 
                     != np.append([-1], h[s][:-1]))
                    [np.argsort(s)]]
        poss = poss[~np.isin(hlist(poss), hist)]
        hist += hlist(poss)
    tot_n_press += n_press

tot_n_press

491

In [631]:
# Part 2
def get_search_values(x,y=None):
    if y is None:
        y = np.empty((1, 0), dtype=int)
    if x.shape[0]:
        return get_search_values(x[1:],
                                 np.column_stack((np.repeat(y, np.ptp(x[0]), axis=0),
                                                  np.tile(np.arange(*x[0]), y.shape[0]))))
    else:
        return y
    
def divide_cols(work):
    pf = [2]
    while pf[-1] <= np.sqrt(np.max(work)):
        while ~np.all(work == (work:=np.where(np.all(np.mod(work, pf[-1]) == 0, 
                                                     axis=0, 
                                                     keepdims=True), 
                                              work//pf[-1], work))):
            pass
        pf += [pf[-1]+1]
        while any([pf[-1] % i == 0 for i in pf[:-1]]):  pf[-1] += 1
    return work
    
def get_min_presses(r_buttons, r_req):
    work = np.append(r_buttons[np.argsort(-np.sum(r_buttons, axis=1))], [r_req], axis=0)
    
    r, c = 0, 0
    while all([i<j for i,j in zip([r,c], r_buttons.shape)]):
        if (y:=np.abs(work[r, c:]))[(z:=np.argmax(y))] == 0:
            r += 1
        else:
            work[:, [c, c+z]] = work[:, [c+z, c]]
            work *= (z:=np.where(np.mod(work[r:r+1], work[r:r+1, c]) != 0, 
                                 work[r:r+1, c], 1))
            work -= np.where(np.expand_dims(np.arange(work.shape[1]) == c, 0), 
                             0, 
                             work[r:r+1] // work[r:r+1, c] * work[:, c:c+1])
            work = divide_cols(work)
            r,c = r+1,c+1
            
    work = work[np.ix_(np.lexsort((np.arange(work.shape[0]), np.sum(work != 0, axis=1) > 1)),
                       np.arange(work.shape[1])[np.any(work != 0, axis=0)])]
    
    work *= np.sign(work[np.min(np.where(work==0, 
                                         work.shape[0]-1, 
                                         np.expand_dims(np.arange(work.shape[0]), 1)), 
                                axis=0), 
                         np.arange(work.shape[1])])
        
    ptr = np.argmax(np.sum(work != 0, axis=1) > 1)
    if (n:=(srows:=work[ptr:-1]).shape[0]):
        workc = np.copy(work)
        srmax = np.full((n, 1), np.nan)
        while np.any(np.isnan(srmax)):
            rmax = np.where((workc[-1:] >= 0) 
                            & np.all(workc[ptr:-1] >= 0, axis=0, keepdims=True),
                            np.where(workc[ptr:-1] == 0, 
                                     np.nan, 
                                     workc[-1:] // np.where(workc[ptr:-1]==0, 1, workc[ptr:-1])),
                            np.nan)
            srmax[ix] = (z:=np.nanmin(rmax[(ix:=np.arange(n)[np.any(~np.isnan(rmax), axis=1) & np.isnan(np.squeeze(srmax, 1))])], 
                                      axis=1, keepdims=True))
            tmult = np.nan_to_num(((x:=np.repeat([srmax], (y:=2**srmax.shape[0]), axis=0)) 
                                  * (z:=np.expand_dims(np.unpackbits(np.arange(y).astype(np.uint8))
                                                       .reshape(y, -1)[:, -srmax.shape[0]:], 
                                                       2)))[np.all(~np.isnan(x) 
                                                                   | (z == 0), axis=(1,2))]).astype(int)
            workc[-1] -= np.min(np.sum(tmult * np.expand_dims(workc[ptr:-1], 0), axis=1), axis=0)
            workc[ptr:-1] = np.where(np.isnan(srmax), workc[ptr:-1], 0)
        srmax = srmax.astype(int)
        
        srmin = np.nanmax(np.column_stack((
                    np.zeros(work.shape[0] - 1 - ptr), 
                    np.where((work[-1:] <= 0) 
                                   & (np.sum(work[ptr:-1] < 0, axis=0, keepdims=True) == 1) 
                                   & (np.sum(work[ptr:-1] > 0, axis=0, keepdims=True) == 0),
                                   np.where(work[ptr:-1] == 0, 
                                            np.nan, 
                                            work[-1:] // np.where(work[ptr:-1]==0, 1, work[ptr:-1])),
                                   np.nan))), 
                          axis=1).astype(int)        
    
        search_ranges = np.column_stack((srmin, srmax + 1))

        return np.min(np.sum(np.append((z:=(work[-1:] 
                                            - np.sum(np.expand_dims((y:=get_search_values(search_ranges)), 2) 
                                                     * np.expand_dims(srows, 0), axis=1)) 
                                        / np.max(work[:ptr], axis=0)).astype(int), 
                                       y, axis=1)[np.all((z >= 0) & (np.mod(z, 1) == 0), axis=1)], axis=1)).item()        
    else:
        return np.sum(work[-1]).item()

In [633]:
for i, (_, buttons, req) in enumerate(manual):
    print(i, get_min_presses(buttons, req))

0 138
1 38
2 52
3 64
4 62
5 107
6 199
7 129
8 78
9 39
10 237
11 81
12 107
13 83
14 111
15 219
16 81
17 74
18 93
19 60
20 93
21 91
22 180
23 201
24 222
25 95
26 66
27 213
28 8
29 191
30 39
31 215
32 26
33 82
34 68
35 60
36 53
37 239
38 56
39 57
40 57
41 193
42 37
43 228
44 76
45 48
46 26
47 63
48 93
49 14
50 42
51 226
52 156
53 67
54 57
55 212
56 206
57 141
58 89
59 269
60 171
61 87
62 77
63 220
64 87
65 45
66 72
67 53
68 58
69 33
70 191
71 74
72 50
73 259
74 74
75 90
76 80
77 55
78 115
79 115
80 275
81 212
82 60
83 149
84 50
85 44
86 73
87 63
88 72
89 128
90 91
91 19
92 147
93 80
94 73
95 263
96 59
97 84
98 142
99 287
100 90
101 55
102 118
103 67
104 74
105 264
106 85
107 11
108 150
109 68
110 144
111 92
112 199
113 32
114 39
115 46
116 11
117 109
118 185
119 56
120 23
121 178
122 317
123 82
124 113
125 236
126 165
127 56
128 147
129 262
130 56
131 232
132 90
133 136
134 100
135 118
136 110
137 201
138 80
139 110
140 195
141 161
142 73
143 126
144 57
145 51
146 109
147 70
148 237
149 2

KeyboardInterrupt: 

In [634]:
_, r_buttons, r_req = manual[172]
r_buttons, r_req

(array([[1, 1, 1, 1, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 1],
        [0, 0, 1, 0, 1, 0, 0, 0],
        [0, 1, 0, 1, 1, 0, 1, 1],
        [0, 1, 0, 1, 0, 0, 1, 1],
        [1, 1, 0, 0, 1, 0, 1, 1],
        [1, 0, 0, 1, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 0, 1, 1],
        [1, 1, 1, 0, 1, 1, 1, 1]]),
 array([30, 56, 41, 36, 31, 17, 56, 56]))

In [636]:
# get_min_presses(r_buttons, r_req)

In [637]:
    work = np.append(r_buttons[np.argsort(-np.sum(r_buttons, axis=1))], [r_req], axis=0)
    r, c = 0, 0
    while all([i<j for i,j in zip([r,c], r_buttons.shape)]):
        if (y:=np.abs(work[r, c:]))[(z:=np.argmax(y))] == 0:
            r += 1
        else:
            work[:, [c, c+z]] = work[:, [c+z, c]]
            work *= (z:=np.where(np.mod(work[r:r+1], work[r:r+1, c]) != 0, 
                                 work[r:r+1, c], 1))
            work -= np.where(np.expand_dims(np.arange(work.shape[1]) == c, 0), 
                             0, 
                             work[r:r+1] // work[r:r+1, c] * work[:, c:c+1])
            work = divide_cols(work)
            r,c = r+1,c+1
    work

array([[ 1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0, -1,  0],
       [ 1,  3, -1,  3, -3, -1,  2,  0],
       [-2, -6,  2, -5,  7,  2, -4,  0],
       [18, 13, -1, 14, 11, -1, -1,  0]])

In [638]:
    work = work[np.ix_(np.lexsort((np.arange(work.shape[0]), np.sum(work != 0, axis=1) > 1)),
                       np.arange(work.shape[1])[np.any(work != 0, axis=0)])]
    work

array([[ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0, -1],
       [ 1,  3, -1,  3, -3, -1,  2],
       [-2, -6,  2, -5,  7,  2, -4],
       [18, 13, -1, 14, 11, -1, -1]])

In [639]:
    work *= np.sign(work[np.min(np.where(work==0, 
                                         work.shape[0]-1, 
                                         np.expand_dims(np.arange(work.shape[0]), 1)), 
                                axis=0), 
                         np.arange(work.shape[1])])

    work

array([[ 1,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  1],
       [ 1,  3, -1,  3, -3, -1, -2],
       [-2, -6,  2, -5,  7,  2,  4],
       [18, 13, -1, 14, 11, -1,  1]])

In [670]:
if (nfree:=work.shape[0] - (ptr:=np.argmax(np.sum(work != 0, axis=1) > 1)) - 1):
    pass

In [668]:
get_search_values = lambda x,y=100: [[i]+j for i in range(y) for j in get_search_values(x-1,y)] if x else [[]]

In [671]:
get_search_values(nfree)

[[0, 0],
 [0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [0, 5],
 [0, 6],
 [0, 7],
 [0, 8],
 [0, 9],
 [0, 10],
 [0, 11],
 [0, 12],
 [0, 13],
 [0, 14],
 [0, 15],
 [0, 16],
 [0, 17],
 [0, 18],
 [0, 19],
 [0, 20],
 [0, 21],
 [0, 22],
 [0, 23],
 [0, 24],
 [0, 25],
 [0, 26],
 [0, 27],
 [0, 28],
 [0, 29],
 [0, 30],
 [0, 31],
 [0, 32],
 [0, 33],
 [0, 34],
 [0, 35],
 [0, 36],
 [0, 37],
 [0, 38],
 [0, 39],
 [0, 40],
 [0, 41],
 [0, 42],
 [0, 43],
 [0, 44],
 [0, 45],
 [0, 46],
 [0, 47],
 [0, 48],
 [0, 49],
 [0, 50],
 [0, 51],
 [0, 52],
 [0, 53],
 [0, 54],
 [0, 55],
 [0, 56],
 [0, 57],
 [0, 58],
 [0, 59],
 [0, 60],
 [0, 61],
 [0, 62],
 [0, 63],
 [0, 64],
 [0, 65],
 [0, 66],
 [0, 67],
 [0, 68],
 [0, 69],
 [0, 70],
 [0, 71],
 [0, 72],
 [0, 73],
 [0, 74],
 [0, 75],
 [0, 76],
 [0, 77],
 [0, 78],
 [0, 79],
 [0, 80],
 [0, 81],
 [0, 82],
 [0, 83],
 [0, 84],
 [0, 85],
 [0, 86],
 [0, 87],
 [0, 88],
 [0, 89],
 [0, 90],
 [0, 91],
 [0, 92],
 [0, 93],
 [0, 94],
 [0, 95],
 [0, 96],
 [0, 97],
 [0, 98],
 [0, 99],
 [1, 0],
 

In [630]:
np.min(np.sum(np.append((z:=(work[-1:] 
                                            - np.sum(np.expand_dims((y:=get_search_values(search_ranges)), 2) 
                                                     * np.expand_dims(srows, 0), axis=1)) 
                                        / np.max(work[:ptr], axis=0)).astype(int), 
                                       y, axis=1)[np.all((z >= 0) & (np.mod(z, 1) == 0), axis=1)], axis=1)).item()

110

In [658]:
tv1 = np.array([[i,j] for i in range(100) for j in range(100)])
# tv1 = np.array([[i,j,k] for i in range(100) for j in range(100) for k in range(100)])
# tv1 = tv1[np.all(tv1 == [[7, 2, 20]], axis=1)]
tv1

array([[ 0,  0],
       [ 0,  1],
       [ 0,  2],
       ...,
       [99, 97],
       [99, 98],
       [99, 99]], shape=(10000, 2))

In [649]:
np.min(np.sum(np.append((z:=(work[-1:] 
                                            - np.sum(np.expand_dims((y:=tv1), 2) 
                                                     * np.expand_dims(srows, 0), axis=1)) 
                                        / np.max(work[:ptr], axis=0)).astype(int), 
                                       y, axis=1)[np.all((z >= 0) & (np.mod(z, 1) == 0), axis=1)], axis=1)).item()

56

In [655]:
tv2 = np.append((z:=(work[-1:] 
                                            - np.sum(np.expand_dims((y:=tv1), 2) 
                                                     * np.expand_dims(srows, 0), axis=1)) 
                                        / np.max(work[:ptr], axis=0)).astype(int), 
                                       y, axis=1)[np.all((z >= 0) & (np.mod(z, 1) == 0), axis=1)]

In [657]:
tv2

array([[17, 10,  0, 11, 14,  0,  3,  1,  0],
       [16,  7,  1,  8, 17,  1,  5,  2,  0],
       [15,  4,  2,  5, 20,  2,  7,  3,  0],
       [17, 10,  0, 10, 13,  0,  3,  3,  1],
       [14,  1,  3,  2, 23,  3,  9,  4,  0],
       [16,  7,  1,  7, 16,  1,  5,  4,  1],
       [15,  4,  2,  4, 19,  2,  7,  5,  1],
       [17, 10,  0,  9, 12,  0,  3,  5,  2],
       [14,  1,  3,  1, 22,  3,  9,  6,  1],
       [16,  7,  1,  6, 15,  1,  5,  6,  2],
       [15,  4,  2,  3, 18,  2,  7,  7,  2],
       [17, 10,  0,  8, 11,  0,  3,  7,  3],
       [14,  1,  3,  0, 21,  3,  9,  8,  2],
       [16,  7,  1,  5, 14,  1,  5,  8,  3],
       [15,  4,  2,  2, 17,  2,  7,  9,  3],
       [17, 10,  0,  7, 10,  0,  3,  9,  4],
       [16,  7,  1,  4, 13,  1,  5, 10,  4],
       [15,  4,  2,  1, 16,  2,  7, 11,  4],
       [17, 10,  0,  6,  9,  0,  3, 11,  5],
       [16,  7,  1,  3, 12,  1,  5, 12,  5],
       [15,  4,  2,  0, 15,  2,  7, 13,  5],
       [17, 10,  0,  5,  8,  0,  3, 13,  6],
       [16

In [651]:
np.min(tv2, axis=0), np.max(tv2, axis=0)

(array([14,  1,  0,  0,  3,  0,  3,  1,  0]),
 array([17, 10,  3, 11, 23,  3,  9, 23, 11]))